## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [2]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

print(tf.__version__)

2.0.0


In [3]:

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test)

(50000, 32, 32, 3)


In [4]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder(categories='auto')
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [7]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(filters=32,
                             kernel_size=(3,3),
                             activation='relu',
                             padding='same',
                             input_shape=(32,32,3)))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(filters=64,
                             kernel_size=(3,3),
                             padding='same',
                             activation='relu',))
classifier.add(BatchNormalization())

classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(10,activation='softmax'))

classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
batch_normalization_3 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)             

In [8]:
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=10)

Train on 50000 samples
Epoch 1/10
50000/50000 [==============================] - 16s 324us/sample - loss: 1.2994 - accuracy: 0.5370
Epoch 2/10
50000/50000 [==============================] - 13s 267us/sample - loss: 0.9043 - accuracy: 0.6803
Epoch 3/10
50000/50000 [==============================] - 14s 275us/sample - loss: 0.7385 - accuracy: 0.7402
Epoch 4/10
50000/50000 [==============================] - 13s 259us/sample - loss: 0.6164 - accuracy: 0.7829- loss:
Epoch 5/10
50000/50000 [==============================] - 13s 268us/sample - loss: 0.5026 - accuracy: 0.8224
Epoch 6/10
50000/50000 [==============================] - 14s 273us/sample - loss: 0.4016 - accuracy: 0.8602- loss: 0.4013 - accuracy: 0.
Epoch 7/10
50000/50000 [==============================] - 14s 278us/sample - loss: 0.3183 - accuracy: 0.8871
Epoch 8/10
50000/50000 [==============================] - 14s 279us/sample - loss: 0.2437 - accuracy: 0.9153
Epoch 9/10
50000/50000 [==============================] - 14s 289us/s

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [14]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[6.0819698e-06, 5.1686311e-06, 4.3947020e-07, 9.9942362e-01,
        3.1923776e-04, 3.6382653e-06, 1.2385724e-05, 1.5960843e-06,
        2.2766336e-04, 2.0741729e-07]], dtype=float32)